# Retrieval Augmented Question & Answering with Amazon Bedrock using LangChain

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

### Context
이전에는 모델이 타이어 교체 방법을 알려주는 것을 보았지만 관련 데이터를 수동으로 제공하고 컨텍스트를 직접 제공해야 했습니다. 우리는 Bedrock에서 사용할 수 있는 모델을 활용하고 교육 중에 학습한 지식과 수동 컨텍스트 제공을 기반으로 질문하는 접근 방식을 탐색했습니다. 이러한 접근 방식은 짧은 문서나 단일 톤 애플리케이션에서는 작동하지만, 모델에 전송된 프롬프트에 모두 들어맞을 수 없는 대규모 기업 문서가 있을 수 있는 기업 수준의 질문 응답으로 확장되지 않습니다. 

### Pattern
RAG(검색 증강 생성)라는 아키텍처를 구현하여 이 프로세스를 개선할 수 있습니다. RAG는 언어 모델 외부(비모수적)에서 데이터를 검색하고 검색된 관련 데이터를 컨텍스트에 추가하여 프롬프트를 강화합니다.

이 노트에서는 질문 응답 패턴에 접근하여 문서를 찾고 활용하여 사용자 질문에 대한 답변을 제공하는 방법을 설명합니다.

### Challenges
- 토큰 한도를 초과하는 대용량 문서 관리 방법
- 질문과 관련된 문서를 찾는 방법

### Proposal
위의 과제에 대해 본 노트북에서는 다음과 같은 전략을 제안합니다.

#### Prepare documents
![Embeddings](./images/Embeddings_lang.png)

질문에 답하기 전에 문서를 처리하고 문서 chunk 색인에 저장해야 합니다.
- 문서를 로드
- 더 작은 청크(chunk)로 처리하고 분할합니다.
- Amazon Bedrock Titan Embeddings 모델을 사용하여 각 청크의 수치 벡터 표현 생성
- 청크와 해당 임베딩을 사용하여 인덱스 생성

#### Ask question
![Question](./images/Chatbot_lang.png)

문서 색인이 준비되면 질문할 준비가 된 것이며, 질문한 질문에 따라 관련 문서를 가져옵니다. 다음 단계가 실행됩니다.
- 입력 질문의 임베딩 생성
- 질문 임베딩과 인덱스의 임베딩을 비교합니다.
- (상위 N) 관련 문서 청크를 가져옵니다.
- 프롬프트에서 컨텍스트의 일부로 해당 청크를 추가합니다.
- Amazon Bedrock의 모델에 프롬프트를 보냅니다.
- 검색된 문서를 기반으로 상황에 맞는 답변을 얻습니다.

## Usecase
#### Dataset
이 아키텍처 패턴을 설명하기 위해 IRS의 문서를 사용합니다. 이 문서에서는 다음과 같은 주제를 설명합니다:
- OID(원래 발행 할인) 상품
- $10,000 이상의 현금 지급을 IRS에 신고
- 고용주 세금 안내

#### Persona
IRS의 작동 방식과 일부 조치가 영향을 미치는지 여부를 이해하지 못하는 일반인의 페르소나를 가정해 보겠습니다.

모델은 문서를 바탕으로 쉬운 언어로 답변하려고 노력할 것입니다.

## Implementation
RAG 접근 방식을 따르기 위해 이 노트북은 RAG와 같은 패턴을 효율적으로 구축할 수 있는 다양한 서비스 및 도구와 통합된 LangChain 프레임워크를 사용하고 있습니다. 우리는 다음 도구를 사용할 것입니다:

- **LLM (Large Language Model)**: Anthropic Claude V1 available through Amazon Bedrock

  이 모델은 문서 청크(chunk)를 이해하고 인간 친화적인 방식으로 답변을 제공하는 데 사용됩니다.
- **Embeddings Model**: Amazon Titan Embeddings available through Amazon Bedrock

  이 모델은 텍스트 문서의 수치 표현을 생성하는 데 사용됩니다.
- **Document Loader**: PDF Loader available through LangChain

  이는 소스에서 문서를 로드할 수 있는 로더입니다. 이 노트북을 위해 로컬 경로에서 샘플 파일을 로드합니다. 이는 기업 내부 시스템에서 문서를 로드하는 로더로 쉽게 대체될 수 있습니다.

- **Vector Store**: FAISS available through LangChain

  이 노트북에서는 이 메모리 내 벡터 chunk를 사용하여 임베딩과 문서를 모두 저장합니다. 엔터프라이즈 환경에서는 이는 AWS OpenSearch, pgVector가 포함된 RDS Postgres, ChromaDB, Pinecone 또는 Weaviate와 같은 영구 chunk로 대체될 수 있습니다.

- **Index**: VectorIndex

  인덱스는 입력 임베딩과 문서 임베딩을 비교하여 관련 문서를 찾는 데 도움이 됩니다.
- **Wrapper**: 인덱스, 벡터 chunk, 임베딩 모델 및 LLM을 래핑하여 사용자로부터 논리를 추상화합니다.

## Setup

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 (필요한 라이브러리가 설치되어 있는지 확인하고) Bedrock에 연결해야 합니다.

설정 작동 방식과 ⚠️ **변경이 필요한지 여부**에 대한 자세한 내용은 [Bedrock boto3 setup notebook](../00_Setup/setup.ipynb) 노트북을 참조하세요.

이 노트북에는 몇 가지 추가 종속성도 필요합니다:

- [FAISS](https://github.com/facebookresearch/faiss), to store vector embeddings
- [PyPDF](https://pypi.org/project/pypdf/), for handling PDF files

In [2]:
! pip list | grep langchain
! pip list | grep opensearch

langchain                            0.0.312
opensearch-py                        2.3.2


In [3]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [4]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print(colored("\n== FM lists ==", "green"))
pprint(bedrock_info.get_list_fm_models())

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Command': 'cohere.command-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'TBD'}


## Configure langchain

LLM 및 임베딩 모델을 인스턴스화하는 것부터 시작합니다. 여기서는 텍스트 생성을 위해 Anthropic Claude를 사용하고 텍스트 삽입을 위해 Amazon Titan을 사용합니다.

참고: Bedrock과 함께 사용 가능한 다른 모델을 선택할 수도 있습니다. 모델을 변경하려면 다음과 같이 `model_id`를 교체하면 됩니다.

`llm = Bedrock(model_id="amazon.titan-tg1-large")`

Available model IDs include:

- `amazon.titan-tg1-large`
- `ai21.j2-grande-instruct`
- `ai21.j2-jumbo-instruct`
- `anthropic.claude-instant-v1`
- `anthropic.claude-v1`

In [5]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

# - create the Anthropic Model
llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs={'max_tokens_to_sample':1000})
bedrock_embeddings = BedrockEmbeddings(client=boto3_bedrock)

In [6]:
llm

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f968da55120>, model_id='anthropic.claude-v2', model_kwargs={'max_tokens_to_sample': 1000})

In [7]:
bedrock_embeddings

BedrockEmbeddings(client=<botocore.client.BedrockRuntime object at 0x7f968da55120>, region_name=None, credentials_profile_name=None, model_id='amazon.titan-embed-text-v1', model_kwargs=None, endpoint_url=None)

## Data Preparation
먼저 문서 저장소를 구축하기 위해 일부 파일을 다운로드해 보겠습니다. 이 예에서는 여기에서 공개 IRS 문서를 사용합니다.[here](https://www.irs.gov/publications).

In [8]:
from urllib.request import urlretrieve

os.makedirs("data", exist_ok=True)
files = [
    "https://www.irs.gov/pub/irs-pdf/p1544.pdf",
    "https://www.irs.gov/pub/irs-pdf/p15.pdf",
    "https://www.irs.gov/pub/irs-pdf/p1212.pdf",
]
for url in files:
    file_path = os.path.join("data", url.rpartition("/")[2])
    urlretrieve(url, file_path)

다운로드한 후 [DirectoryLoader from PyPDF available under LangChain](https://python.langchain.com/en/latest/reference/modules/document_loaders.html)를 사용하여 문서를 로드하고 더 작은 청크(chunk)로 나눌 수 있습니다.

참고: 검색된 문서/텍스트는 질문에 대답하기에 충분한 정보를 포함할 만큼 커야 합니다. 하지만 LLM 프롬프트에 들어갈 만큼 충분히 작습니다. 또한 임베딩 모델에는 입력 토큰 길이가 512개 토큰으로 제한되어 있으며 이는 대략 2000자까지 변환됩니다. 이 사용 사례를 위해 우리는 [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html)를 사용하여 100자가 겹치는 대략 1000자의 청크를 생성합니다.

In [9]:
import numpy as np
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("./data/")

documents = loader.load()
# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 100,
)
docs = text_splitter.split_documents(documents)

In [10]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)
avg_char_count_pre = avg_doc_length(documents)
avg_char_count_post = avg_doc_length(docs)
print(f'Average length among {len(documents)} documents loaded is {avg_char_count_pre} characters.')
print(f'After the split we have {len(docs)} documents more than the original {len(documents)}.')
print(f'Average length among {len(docs)} documents (after split) is {avg_char_count_post} characters.')

Average length among 73 documents loaded is 5850 characters.
After the split we have 503 documents more than the original 73.
Average length among 503 documents (after split) is 910 characters.


우리는 3개의 PDF 문서를 500개 이하의 작은 청크(chunk)로 분할했습니다. 

이제 해당 청크 중 하나에 대한 샘플 임베딩이 어떻게 보이는지 확인할 수 있습니다.

In [11]:
sample_embedding = np.array(bedrock_embeddings.embed_query(docs[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

Sample embedding of a document chunk:  [ 0.11621094  0.06494141 -0.23730469 ...  0.11962891 -0.29882812
 -0.27929688]
Size of the embedding:  (1536,)


유사한 패턴에 따라 전체 코퍼스에 대해 임베딩을 생성하고 벡터 chunk에 저장할 수 있습니다. 

이는 임베딩 모델과 문서를 입력받아 전체 벡터 저장소를 생성하는 LangChain 내부의 [FAISS](https://github.com/facebookresearch/faiss)  구현을 사용하여 쉽게 수행할 수 있습니다. [LangChain](https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/faiss.html)를 사용하면 프롬프트 생성, 쿼리 임베딩 가져오기, 관련 문서 샘플링 및 LLM 호출과 같은 대부분의 무거운 작업을 추상화할 수 있습니다. [VectorStoreIndexWrapper](https://python.langchain.com/docs/modules/data_connection/retrievers)가 이를 도와줍니다.

**⚠️⚠️⚠️ NOTE: it might take few minutes to run the following cell ⚠️⚠️⚠️**

In [12]:
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import FAISS
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

vectorstore_faiss = FAISS.from_documents(
    docs,
    bedrock_embeddings,
)

wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss)

## Question Answering

이제 벡터 저장소가 준비되었으므로 질문을 시작할 수 있습니다.

In [13]:
wrapper_store_faiss

VectorStoreIndexWrapper(vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x7f96874874f0>)

In [18]:
query = "\n\nHuman:Is it possible that I get sentenced to jail due to failure in filings?\n\nAssistant:"

첫 번째 단계는 문서와 비교할 수 있도록 쿼리 임베딩을 만드는 것입니다.

In [19]:
query_embedding = vectorstore_faiss.embedding_function(query)
np.array(query_embedding)

array([ 0.25976562, -0.22558594,  0.26953125, ...,  0.390625  ,
       -0.0703125 ,  0.30859375])

이 쿼리 임베딩을 사용하여 관련 문서를 가져올 수 있습니다. 이제 쿼리는 임베딩으로 표시되어 가장 관련성이 높은 정보를 제공하는 데이터 저장소에 대해 쿼리의 유사성 검색을 수행할 수 있습니다.

In [20]:
relevant_documents = vectorstore_faiss.similarity_search_by_vector(query_embedding)
print(f'{len(relevant_documents)} documents are fetched which are relevant to the query.')
print('----')
for i, rel_doc in enumerate(relevant_documents):
    print_ww(f'## Document {i+1}: {rel_doc.page_content}.......')
    print('---')

4 documents are fetched which are relevant to the query.
----
## Document 1: clerks of federal or state courts are discussed
later under Bail received by court clerks.
However, you do not have to file Form 8300
if the transaction is not related to your trade or
business. For example, if you own a jewelry
store and sell your personal automobile for
more than $10,000 in cash, you would not sub-
mit a Form 8300 for that transaction.
Transaction defined.  A “transaction” occurs
when:
Goods, services, or property are sold;
Property is rented;
Cash is exchanged for other cash;
A contribution is made to a trust or escrow
account;
A loan is made or repaid; or
Cash is converted to a negotiable instru-
ment, such as a check or a bond.
Person defined.  A “person” includes an indi-
vidual, a company, a corporation, a partnership,
an association, a trust, or an estate.
Exempt organizations, including employee
plans, are also “persons.” However, exempt or-
ganizations do not have to file Form 8300 f

이제 관련 문서가 있으므로 LLM을 사용하여 이러한 문서를 기반으로 답변을 생성할 차례입니다.

유사성 검색 결과를 기반으로 검색된 관련 문서와 함께 초기 프롬프트를 사용합니다. 그런 다음 이를 결합하여 모델에 피드백하여 결과를 얻는 프롬프트를 생성합니다. 이 시점에서 우리 모델은 매뉴얼에 설명된 대로 특정 자동차의 타이어를 교체할 수 있는 방법에 대한 고도의 정보를 제공해야 합니다.

LangChain은 이를 쉽게 수행할 수 있는 방법에 대한 추상화를 제공합니다.

### Quick way
Vector Store를 둘러싸서 LLM 입력을 받는 LangChain에서 제공하는 Wrapper를 사용할 수 있습니다. 이 Wrapper는 뒤에서 다음 단계를 수행합니다:
- 질문을 입력
- 질문 임베딩 생성
- 관련 문서 가져오기
- 프롬프트에 문서와 질문을 채워 넣습니다.
- 프롬프트로 모델을 호출하고 사람이 읽을 수 있는 방식으로 답변을 생성합니다.

In [21]:
answer = wrapper_store_faiss.query(question=query, llm=llm)
print_ww(answer)

 Based on the information provided, it is possible to be sentenced to jail for willful failure to
file Form 8300 or for willfully filing a false or fraudulent Form 8300. The passage states:

"There are criminal penalties for:
- Willful failure to file Form 8300,
- Willfully filing a false or fraudulent Form 8300,
- Stopping or trying to stop Form 8300 from being filed, and
- Setting up, helping to set up, or trying to set up a transaction in a way that would make it seem
unnecessary to file Form 8300.

If you willfully fail to file Form 8300, you can be fined up to $250,000 for individuals ($500,000
for corporations) or sentenced to up to 5 years in prison, or both."

So yes, it is possible to be sentenced to jail for up to 5 years for willful failure to properly
file Form 8300. The criminal penalties apply for intentional disregard of the reporting
requirements.


Let's ask a different question:

In [22]:
query_2 = "\n\nHuman:What is the difference between market discount and qualified stated interest\n\nAssistant:"

In [23]:
answer_2 = wrapper_store_faiss.query(question=query_2, llm=llm)
print_ww(answer_2)



- Market Discount - This refers to the difference between the price at which you purchased a bond
and its stated redemption price at maturity. Market discount arises when the bond trades at a
discount to its par value due to an increase in interest rates since the bond was issued. The market
discount is considered taxable income when the bond matures or is sold.

- Qualified Stated Interest - This refers to the stated interest rate on the bond that is considered
taxable as ordinary income each year. The stated interest rate on the bond at the time of issue is
considered qualified stated interest. This interest income is taxed annually even if the bond was
purchased for an amount less than its par value.

So in summary:

- Market discount is the difference between the purchase price and par value of a bond, which is
taxed at maturity or sale.

- Qualified stated interest is the annual interest paid on the bond's face value, which is taxed
annually as ordinary income.

The key differen

### Customisable option
위 시나리오에서는 질문에 대한 상황 인식 답변을 빠르고 쉽게 얻을 수 있는 방법을 탐색했습니다. 이제 문서를 가져오는 방법을 사용자 정의할 수 있는 [RetrievalQA](https://python.langchain.com/en/latest/modules/chains/index_examples/Vector_db_qa.html)의 도움으로 더 사용자 정의 가능한 옵션을 살펴보겠습니다. `chain_type` 매개변수를 사용하여 프롬프트에 추가해야 합니다. 또한 검색해야 하는 관련 문서 수를 제어하려면 아래 셀에서 'k' 매개변수를 변경하여 다른 출력을 확인하세요. 많은 시나리오에서 LLM이 답변을 생성하는 데 사용한 소스 문서가 무엇인지 알고 싶을 수 있습니다. LLM 프롬프트의 컨텍스트에 추가된 문서를 반환하는 `return_source_documents`를 사용하여 출력에서 해당 문서를 가져올 수 있습니다. 'RetrievalQA'를 사용하면 모델에 특정한 사용자 정의 [프롬프트 템플릿](https://python.langchain.com/en/latest/modules/prompts/prompt_templates/getting_started.html)을 제공할 수도 있습니다.

참고: 이 예에서는 Amazon Bedrock에서 LLM으로 Anthropic Claude를 사용하고 있습니다. 이 특정 모델은 입력이 `Human:` 아래에 제공되고 모델이 `Assistant:` 다음에 출력을 생성하도록 요청하는 경우 가장 잘 수행됩니다. 아래 셀에는 LLM이 고정 상태를 유지하고 컨텍스트 외부에서 응답하지 않도록 프롬프트를 제어하는 방법의 예가 나와 있습니다.

In [25]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Human: Use the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_faiss.as_retriever(
        search_type="similarity", search_kwargs={"k": 3}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)
query = "Is it possible that I get sentenced to jail due to failure in filings?"
result = qa({"query": query})
print_ww(result['result'])

 Based on the information provided, it is possible to be sentenced to jail for certain failures
related to filing Form 8300. Specifically:

- There are criminal penalties for willfully failing to file Form 8300, willfully filing a false or
fraudulent Form 8300, stopping or trying to stop Form 8300 from being filed, or setting up
transactions to avoid filing.

- If you willfully fail to file Form 8300, you can be fined up to $250,000 (individuals) or $500,000
(corporations), sentenced to up to 5 years in prison, or both.

So in summary, yes, it is possible to be sentenced to jail due to willful failures related to filing
Form 8300 correctly and completely. The criminal penalties apply for egregious violations, not
simple mistakes. But intentionally not filing or falsifying the form could potentially lead to jail
time.


In [26]:
def show_context_used(context_list):
    for idx, context in enumerate(context_list):
        print("-----------------------------------------------")                
        print(f"{idx+1}. context to be fed into FM(e.g. Claude-v2)")
        print("-----------------------------------------------")        
        print_ww(context.page_content)        
show_context_used(result['source_documents'])        

-----------------------------------------------
1. context to be fed into FM(e.g. Claude-v2)
-----------------------------------------------
There are civil penalties for failure to:
File a correct Form 8300 by the date it is
due, and
Provide the required statement to those
named in the Form 8300.
If you intentionally disregard the requirement
to file a correct Form 8300 by the date it is due,
the penalty is the greater of:
1.$25,000, or
2.The amount of cash you received and
were required to report (up to $100,000).
There are criminal penalties for:
Willful failure to file Form 8300,
Willfully filing a false or fraudulent Form
8300,
Stopping or trying to stop Form 8300 from
being filed, and
Setting up, helping to set up, or trying to
set up a transaction in a way that would
make it seem unnecessary to file Form
8300.
If you willfully fail to file Form 8300, you can
be fined up to $250,000 for individuals
RECORDS($500,000 for corporations) or sentenced to up
to 5 years in prison, or bot

## Conclusion
RAG(검색 증강 생성)에 관한 이 모듈을 완료한 것을 축하합니다! 이는 대규모 언어 모델의 성능과 검색 방법의 정확성을 결합하는 중요한 기술입니다. 검색된 관련 사례로 생성을 강화함으로써 우리가 받은 응답은 더욱 일관되고 일관되며 근거가 있게 됩니다. 당신은 이 혁신적인 접근 방식을 배우는 것에 자부심을 느껴야 합니다. 나는 당신이 얻은 지식이 창의적이고 매력적인 언어 생성 시스템을 구축하는 데 매우 유용할 것이라고 확신합니다. 잘하셨어요!

위의 RAG 기반 질문 응답 구현에서 우리는 다음 개념과 Amazon Bedrock 및 LangChain 통합을 사용하여 이를 구현하는 방법을 탐색했습니다.

- 문서 로드 및 임베딩 생성을 통해 벡터 저장소 생성
- 질문에 대한 문서 검색
- LLM에 대한 입력으로 사용되는 프롬프트 준비
- 인간 친화적인 방식으로 답변 제시

### Take-aways
- 다양한 벡터 스토어를 실험해 보세요.
- Amazon Bedrock에서 사용 가능한 다양한 모델을 활용하여 대체 출력을 확인하세요.
- 임베딩 및 문서 청크의 영구 저장과 같은 옵션 탐색
- 엔터프라이즈 데이터 저장소와의 통합

# Thank You